In [1]:
import numpy as np
import pandas as pd
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph
from pgmpy.inference import BeliefPropagation

In [2]:
np.random.rand(4)

array([0.76225608, 0.864833  , 0.56522779, 0.96563802])

## Task 2 

### 2.1

In [3]:
G = FactorGraph() ## Create FactorGraph object
#G.add_nodes_from()
###############################
#   TODO: Define factor functions
###############################
f_1 = DiscreteFactor(['S1'],[2],[8.5,1.5])
f_2 = DiscreteFactor(['S1','E1'],[2,2],[1.,2.,0.,5.])

###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G.add_nodes_from(['E1', 'S1'])  ## Add random variables 
G.add_factors(f_1,f_2)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G.add_edges_from([('S1',f_1), ('S1',f_2), ('E1',f_2)])


### 2.2

In [4]:
bp = BeliefPropagation(G)
###############################
#   TODO: Compute the marginal probability
###############################
margin = bp.query(variables=['S1'],evidence=[])
#dir(margin)
print(margin)

Eliminating: E1: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 338.74it/s]

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |   25.5000 |
+-------+-----------+
| S1(1) |    7.5000 |
+-------+-----------+


### 2.3

#### S1 = 0 maximizes the marginal probability P(S1)

### 2.4

In [5]:
bp1 = BeliefPropagation(G)
###############################
#   TODO: Compute the marginal probability
###############################
margin1 = bp1.query(variables=['S1'],evidence={'E1':1},)
#dir(margin)
print(margin1)

0it [00:00, ?it/s]

+-------+-----------+
| S1    |   phi(S1) |
+=======+===========+
| S1(0) |   17.0000 |
+-------+-----------+
| S1(1) |    7.5000 |
+-------+-----------+


### 2.5

S1 = 0 maximizes the probability P(S1|E1=1)

### 2.6

2.2:

P(S1=0) = f(S1=0)*(g(S1=0,E1=0)+g(S1=0,E1=1))= 25.5

P(S1=1) = f(S1=1)*(g(S1=1,E1=0)+g(S1=1,E1=1))= 7.5

2.4:

P(S1=0|E1=1) = f(S1=0)*g(S1=0,E1=1) = 17

P(S1=1|E1=1) = f(S1=1)*g(S1=1,E1=1) = 7.5

## Task 3

In [6]:
ATTACK_EVENTS_MAP = {
    'Scan':1,
    'Login':2,
    'Sensitive_URI':3,
    'New_Kernel_Module':4,
    'DNS_Tunneling':5
}
ATTACK_STATES_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}




### Task 3.0 

In [7]:
event_review = open('event_review.txt', 'r') 
Lines = event_review.readlines() 
f_gen=np.zeros((5,11))
for line in Lines: 
    temp = line.split(" //")
    temp[0] = temp[0][7:]
    temp[1] = temp[1].strip('/')
    temp[1] = temp[1].strip('\n')
    temp[1] = temp[1][21:]
    temp[1] = temp[1].strip(" ")
    #print(ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1)
    f_gen[ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1] = f_gen[ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1] +1.0
for i in range(5):
    f_gen[i,:]=f_gen[i,:]/f_gen[i,:].sum()
f_gen

array([[0.936     , 0.064     , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.55333333, 0.        , 0.        , 0.        , 0.44666667,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.875     , 0.        , 0.        , 0.        , 0.        ,
        0.125     , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.02      , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.98      , 0.        ,
        0.        ]])

### Task 3.1

In [8]:
attack_sequences = open('attack_sequences.txt', 'r') 
Lines = attack_sequences.readlines()
pattern={}
total = 0
for line in Lines:
    #print(line)
    #line.rstrip('\n')
    temp = line.rstrip(' \n').split(" ")
    #print(temp)
    for i in range(len(temp)-2):
        total+=1
        key = temp[i]+'->'+temp[i+1]+'->'+temp[i+2]
        if(key in pattern):
            pattern[key]=pattern[key]+1
        else:
            pattern[key]=1
maximum = max(pattern, key=pattern.get)
possibility = pattern[maximum]/total
print("Most Common Event Sequence is ",maximum,"\nthe Probability is", possibility)

Most Common Event Sequence is  Scan->Sensitive_URI->New_Kernel_Module 
the Probability is 0.07147962830593281


In [9]:
ca = [maximum]
cb = ["c"]
cc = ['persistence']
cd  = [possibility]
table = pd.DataFrame({"Most Common Event Sequence": ca, "Factor Function": cb,"Attack States": cc, "Probability": cd})


table

,Most Common Event Sequence,Factor Function,Attack States,Probability
0,Scan->Sensitive_URI->New_Kernel_Module,c,persistence,0.07148


In [10]:
attack_sequences = open('attack_sequences.txt', 'r') 
Lines = attack_sequences.readlines()
patternr={}
for line in Lines:
    #print(line)
    #line.rstrip('\n')
    temp = line.rstrip(' \n').split(" ")
    #print(temp)
    for i in range(len(temp)-2):
        if(temp[i]==temp[i+1] and temp[i+2]==temp[i+1]):
            key = temp[i]+'->'+temp[i+1]+'->'+temp[i+2]
            if(key in patternr):
                patternr[key]=patternr[key]+1
            else:
                patternr[key]=1
print(patternr)
maximumr = max(patternr, key=patternr.get)
possibilityr = patternr[maximumr]/total
print(patternr[maximumr])
print("Most frequent repetitive Event Sequence is ",maximumr,"\nthe Probability is", possibilityr)

{'Sensitive_URI->Sensitive_URI->Sensitive_URI': 186, 'Login->Login->Login': 7, 'Scan->Scan->Scan': 25, 'New_Kernel_Module->New_Kernel_Module->New_Kernel_Module': 25, 'DNS_Tunneling->DNS_Tunneling->DNS_Tunneling': 15}
186
Most frequent repetitive Event Sequence is  Sensitive_URI->Sensitive_URI->Sensitive_URI 
the Probability is 0.06647605432451752


In [11]:
ca = [maximumr]
cb = ["r"]
cc = ['privilege escalation']
cd  = [possibilityr]
table = pd.DataFrame({"Most Common Event Sequence": ca, "Factor Function": cb,"Attack States": cc, "Probability": cd})

table

,Most Common Event Sequence,Factor Function,Attack States,Probability
0,Sensitive_URI->Sensitive_URI->Sensitive_URI,r,privilege escalation,0.066476


### Task 3.2  You will have to submit the graph you draw through Compass

### Task 3.3

In [12]:
event_review = open('event_review.txt', 'r') 
Lines = event_review.readlines() 
f_gen=np.zeros((5,11))
for line in Lines: 
    temp = line.split(" //")
    temp[0] = temp[0][7:]
    temp[1] = temp[1].strip('/')
    temp[1] = temp[1].strip('\n')
    temp[1] = temp[1][21:]
    temp[1] = temp[1].strip(" ")
    #print(ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1)
    f_gen[ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1] = f_gen[ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1] +1.0
for i in range(5):
    f_gen[i,:]=f_gen[i,:]/f_gen[i,:].sum()

### Task 3.4

#### a. At every time point, provide the marginal probability of each state (Since we have 9 time points and 11 possible states, you should provide 99 probability values here)

#### b. At every time point, provide the most probable state

### Task 3.5

In [ ]:
ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}

### Task 3.6

#### Indicate the earliest stage in which your model should recommend stopping the attack

### Task 3.7

#### a. Judge whether the most probable states for $s_1-s_6,s_8,s_9$ remain the same as Task3.2
#### b. State the reason for your judgement

In [ ]:
event_review = open('event_review.txt', 'r') 
Lines = event_review.readlines() 
f_gen=np.zeros((5,11))
for line in Lines: 
    temp = line.split(" //")
    temp[0] = temp[0][7:]
    temp[1] = temp[1].strip('/')
    temp[1] = temp[1].strip('\n')
    temp[1] = temp[1][21:]
    temp[1] = temp[1].strip(" ")
    #print(ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1)
    f_gen[ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1] = f_gen[ATTACK_EVENTS_MAP[temp[0]]-1,ATTACK_STATES_MAP[temp[1]]-1] +1.0
for i in range(5):
    f_gen[i,:]=f_gen[i,:]/f_gen[i,:].sum()

In [ ]:
f_c=[0.07147962830593281,0,0,0,0,0,0,0,0,0,0]
f_r=[0.06647605432451752,0,0,0,0,0,0,0,0,0,0]

In [ ]:
G1 = FactorGraph() ## Create FactorGraph object
###############################
#   TODO: Define factor functions
###############################
f_1 = DiscreteFactor(['E1','S1'],[1,11],f_gen[0])
#f_2 = DiscreteFactor(['E2','S2'],[1,11],f_gen[1])
f_3 = DiscreteFactor(['E3','S3'],[1,11],f_gen[2])
f_4 = DiscreteFactor(['E4','S4'],[1,11],f_gen[2])
f_5 = DiscreteFactor(['E5','S5'],[1,11],f_gen[2])
f_6 = DiscreteFactor(['E6','S6'],[1,11],f_gen[3])
#f_7 = DiscreteFactor(['E7','S7'],[1,11],f_gen[4])
#f_8 = DiscreteFactor(['E8','S8'],[1,11],f_gen[4])
#f_9 = DiscreteFactor(['E9','S9'],[1,11],f_gen[4])

r = DiscreteFactor(['E3','E4','E5','S5'],[1,1,1,11],f_r)
c = DiscreteFactor(['E1','E3','E6','S6'],[1,1,1,11],f_c) 
###############################
#   TODO: Add random variables
#         and factor functions 
###############################
G1.add_nodes_from(['E1','E3','E4','E5','E6','S1','S3','S4','S5','S6'])  ## Add random variables 
G1.add_factors(f_1,f_3,f_4,f_5,f_6,r,c)     ## Add factor functions

###############################
#   TODO: Add the edges for random 
#   variables and factor functions
###############################
G1.add_edges_from([('E1',f_1),('S1',f_1),('E3',f_3),('S3',f_3),('E4',f_4),('S4',f_4),('E5',f_5),('S5',f_5),('E6',f_6),('S6',f_6),('E1',c),('E3',c),('E6',c),('S6',c),('E3',r),('E4',r),('E5',r),('S5',r)])

###############################
#   TODO: Do the inference
###############################
bp1 = BeliefPropagation(G1)

In [ ]:
G2 = FactorGraph()
f_2 = DiscreteFactor(['E2','S2'],[1,11],f_gen[1])
G2.add_nodes_from(['E2','S2'])
G2.add_factors(f_2)
G2.add_edges_from([('E2',f_2),('S2',f_2)])
bp2 = BeliefPropagation(G2)   

In [ ]:
G4 = FactorGraph()
f_8 = DiscreteFactor(['E8','S8'],[1,11],f_gen[4])
G4.add_nodes_from(['E8','S8'])
G4.add_factors(f_8)
G4.add_edges_from([('E8',f_8),('S8',f_8)])
bp4 = BeliefPropagation(G4)    

In [ ]:
G5 = FactorGraph()
f_9 = DiscreteFactor(['E9','S9'],[1,11],f_gen[4])
G5.add_nodes_from(['E9','S9'])
G5.add_factors(f_9)
G5.add_edges_from([('E9',f_9),('S9',f_9)])
bp5 = BeliefPropagation(G5) 

In [ ]:
margin_s1=bp1.query(variables=['S1']).normalize(inplace=False)
print(margin_s1)
margin_s3=bp1.query(variables=['S3']).normalize(inplace=False)
print(margin_s3)
margin_s4=bp1.query(variables=['S4']).normalize(inplace=False)
print(margin_s4)
margin_s5=bp1.query(variables=['S5']).normalize(inplace=False)
print(margin_s5)
margin_s6=bp1.query(variables=['S6']).normalize(inplace=False)
print(margin_s6)

In [ ]:
margin_s2=bp2.query(variables=['S2']).normalize(inplace=False)
print(margin_s2)

In [ ]:
margin_s8=bp4.query(variables=['S8']).normalize(inplace=False)
print(margin_s8)

In [ ]:
margin_s9=bp5.query(variables=['S9']).normalize(inplace=False)
print(margin_s9)

#### 3.7 a)  Based on the calculated marginal probability for S1-S6,S8,S9, we infer the most probable hidden state are as follows: t=1,Benign; t=2, Benign; t=3, Benign; t=4, Benign; t=5,Benign; t=6,Benign; t=8, Extracton; t=99,Extraction
#### b) The most probable hidden states for S1-S6,S8,S9 remain the same. Because the E7 and S7 consist a independent factor graph( the graph have no connection with any other factor graphs), which means that S1-S6,S8,S9 are independent of E7 and S7. So delete E7 and S7 will not affect the S1-S6,S8,S9

### Task 3.8

#### a. Draw an HMM model for the attack scenario given the provided states and events.
#### b. What parameters are needed for this HMM model to work?
#### c. Give an example of an advantage of the FG over the HMM model.